<a href="https://colab.research.google.com/github/marianatiele/etl_aws_combustiveis/blob/main/Notebook_Silver_To_Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Camada Silver**
Nesta camada os dados são tratados e enviados para o Lakehouse Silver, para serem utilizados na camada Gold

## **Baixando o Pyspark**

In [66]:
!pip install findspark

In [67]:
!pip install pyngrok

In [68]:
# instalar o Spark
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  # Instalação do Java

# Baixar a versão do Spark desejada (Spark 3.5.4 com Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz

# Extrair os arquivos do Spark
!tar xf spark-3.5.4-bin-hadoop3.tgz


## **Importando as Bibliotecas**

In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyngrok import ngrok
import findspark
import os

In [70]:
# configurar as variáveis de ambiente

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

aws_access_key_id = "SUA SENHA AQUI"
aws_secret_access_key = "SUA SENHA AQUI"
region_name = "SUA REGIÃO AQUI"

In [71]:
findspark.init()

In [72]:
spark = SparkSession.builder \
    .appName("S3Integration") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1") \
    .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

In [73]:
dados = spark.read.csv('s3a://datalake-combustiveis-br/Bronze/precos-gasolina-etanol-01.csv', header=True, sep=';')

In [74]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+-----------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|Complemento|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+-----------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|       NULL|CENTRO|62010-000|          GASOLINA| 01/01/2025|       6,29|         NULL|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|       NULL|CENTRO|62010-000|GASOLINA ADITIVADA| 01/01/2025|       6,49|         NULL|

In [75]:
dados_2 = spark.read.csv('s3a://datalake-combustiveis-br/Bronze/precos-gasolina-etanol-02.csv', header=True, sep=';')
dados_3 = spark.read.csv('s3a://datalake-combustiveis-br/Bronze/precos-gasolina-etanol-03.csv', header=True, sep=';')

In [76]:
dados_2.show(2)

+------+------+---------------+--------------------+-------------------+--------------------+------+-----------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|      Municipio|             Revenda|               CNPJ|                Nome|Numero|Complemento|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------------+--------------------+-------------------+--------------------+------+-----------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|       NULL|CENTRO|69980-000|          GASOLINA| 03/02/2025|       7,27|         NULL|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|       NULL|CENTRO|69980-000|GASOLINA ADITIVADA| 03/02/2

In [77]:
dados_3.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+-----------+--------------+---------+--------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|Complemento|        Bairro|      Cep| Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+-----------+--------------+---------+--------+-----------+-----------+-------------+--------------+--------+
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|       NULL|JARDIM ZULMIRA|18061-000|GASOLINA| 03/03/2025|       5,65|         NULL|    R$ / litro|  BRANCA|
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|       NULL|JARDIM ZULMIRA|18061-000|  ETANOL| 03/03/2025|       3,78|         NULL|    R$ / l

In [78]:
len(dados.columns), dados.count(), len(dados_2.columns), dados_2.count(), len(dados_3.columns), dados_3.count()

(16, 46509, 16, 44290, 16, 51685)

# Tratamento dos dados

In [79]:
dados = dados.withColumnRenamed('Valor_ Compra', 'Valor_Compra')

In [80]:
colunas = dados.columns
colunas[13]

'Valor_Compra'

In [81]:
dados_2 = (dados_2.withColumnRenamed('Regiao - Sigla', colunas[0])
    .withColumnRenamed('Estado - Sigla', colunas[1])
    .withColumnRenamed('CNPJ da Revenda', colunas[4])
    .withColumnRenamed('Nome da Rua', colunas[5])
    .withColumnRenamed('Numero Rua', colunas[6])
    .withColumnRenamed('Data da Coleta', colunas[11])
    .withColumnRenamed('Valor de Venda', colunas[12])
    .withColumnRenamed('Valor de Compra', colunas[13])
    .withColumnRenamed('Unidade de Medida', colunas[14])
)



In [82]:
dados_3 = (dados_3.withColumnRenamed('Regiao - Sigla', colunas[0])
    .withColumnRenamed('Estado - Sigla', colunas[1])
    .withColumnRenamed('CNPJ da Revenda', colunas[4])
    .withColumnRenamed('Nome da Rua', colunas[5])
    .withColumnRenamed('Numero Rua', colunas[6])
    .withColumnRenamed('Data da Coleta', colunas[11])
    .withColumnRenamed('Valor de Venda', colunas[12])
    .withColumnRenamed('Valor de Compra', colunas[13])
    .withColumnRenamed('Unidade de Medida', colunas[14])
)

In [83]:
dados = dados.withColumnRenamed('Nome', 'RUA')
dados_2 = dados_2.withColumnRenamed('Nome', 'RUA')
dados_3 = dados_3.withColumnRenamed('Nome', 'RUA')

In [84]:
# Decimal tipo americano
dados = dados.withColumn("Valor_Venda", regexp_replace(col("Valor_Venda"), ",", ".").cast("float"))
dados_2 = dados_2.withColumn("Valor_Venda", regexp_replace(col("Valor_Venda"), ",", ".").cast("float"))
dados_3 = dados_3.withColumn("Valor_Venda", regexp_replace(col("Valor_Venda"), ",", ".").cast("float"))

dados = dados.withColumn("Numero", regexp_replace(col("Numero"), " ", "S/N"))
dados_2 = dados_2.withColumn("Numero", regexp_replace(col("Numero"), " ", "S/N"))
dados_3 = dados_3.withColumn("Numero", regexp_replace(col("Numero"), " ", "S/N"))


In [85]:
# Apagar coluna inteira null
dados = dados.drop('Valor_Compra')
dados_2 = dados_2.drop('Valor_Compra')
dados_3 = dados_3.drop('Valor_ Compra')

dados = dados.drop('Complemento')
dados_2 = dados_2.drop('Complemento')
dados_3 = dados_3.drop('Complemento')

In [86]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                 RUA|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|          GASOLINA| 01/01/2025|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|GASOLINA ADITIVADA| 01/01/2025|       6.49|    R$ / litro|  RAIZEN|
+------+------+---------+--------------------+-------------------+--------------------+------+------+----

In [87]:
dados_2.show(2)

+------+------+---------------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|      Municipio|             Revenda|               CNPJ|                 RUA|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|CENTRO|69980-000|          GASOLINA| 03/02/2025|       7.27|         NULL|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|CENTRO|69980-000|GASOLINA ADITIVADA| 03/02/2025|       7.73|         NULL|    R$ / litro|   VIBRA|
+----

In [88]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                 RUA|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|          GASOLINA| 01/01/2025|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|GASOLINA ADITIVADA| 01/01/2025|       6.49|    R$ / litro|  RAIZEN|
+------+------+---------+--------------------+-------------------+--------------------+------+------+----

In [89]:
dados.select('Numero', 'Bairro').where(dados.Numero == 'S/N').show(2)

+------+----------+
|Numero|    Bairro|
+------+----------+
|   S/N|INDUSTRIAL|
|   S/N|INDUSTRIAL|
+------+----------+
only showing top 2 rows



In [90]:
dados_2.select('Numero', 'Bairro').where(dados_2.Numero == 'S/N').show(2)

+------+------+
|Numero|Bairro|
+------+------+
|   S/N| ARAXÁ|
|   S/N| ARAXÁ|
+------+------+
only showing top 2 rows



In [93]:
dados_2.select('Data_Coleta', 'Valor_Venda').show(2)

+-----------+-----------+
|Data_Coleta|Valor_Venda|
+-----------+-----------+
| 03/02/2025|       7.27|
| 03/02/2025|       7.73|
+-----------+-----------+
only showing top 2 rows



In [96]:
dados = dados.select(
'Regiao'.upper(),
 'Estado'.upper(),
 'Municipio'.upper(),
 'Revenda'.upper(),
 'CNPJ'.upper(),
 'Rua'.upper(),
 'Numero'.upper(),
 'Bairro'.upper(),
 'Cep'.upper(),
 'Produto'.upper(),
 'Data_Coleta'.upper(),
 'Valor_Venda'.upper(),
 'Unidade_Medida'.upper(),
 'Bandeira'.upper()
)


In [97]:
dados_2 = dados_2.select(
'Regiao'.upper(),
 'Estado'.upper(),
 'Municipio'.upper(),
 'Revenda'.upper(),
 'CNPJ'.upper(),
 'Rua'.upper(),
 'Numero'.upper(),
 'Bairro'.upper(),
 'Cep'.upper(),
 'Produto'.upper(),
 'Data_Coleta'.upper(),
 'Valor_Venda'.upper(),
 'Unidade_Medida'.upper(),
 'Bandeira'.upper()
)


In [100]:
dados_3 = dados_3.select(
'Regiao'.upper(),
 'Estado'.upper(),
 'Municipio'.upper(),
 'Revenda'.upper(),
 'CNPJ'.upper(),
 'Rua'.upper(),
 'Numero'.upper(),
 'Bairro'.upper(),
 'Cep'.upper(),
 'Produto'.upper(),
 'Data_Coleta'.upper(),
 'Valor_Venda'.upper(),
 'Unidade_Medida'.upper(),
 'Bandeira'.upper()
)


In [101]:
display(dados.show(5))

+------+------+---------+--------------------+-------------------+--------------------+------+----------------+---------+------------------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                 RUA|NUMERO|          BAIRRO|      CEP|           PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+----------------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|          CENTRO|62010-000|          GASOLINA| 01/01/2025|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|          CENTRO|62010-000|GASOLINA ADITIVADA| 01/01/2025|       6.49|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.9

None

In [102]:
display(dados_2.show(5))

+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|      MUNICIPIO|             REVENDA|               CNPJ|                 RUA|NUMERO|    BAIRRO|      CEP|           PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|          GASOLINA| 03/02/2025|       7.27|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|GASOLINA ADITIVADA| 03/02/2025|       7.73|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...| 01

None

In [103]:
display(dados_3.show(5))

+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                 RUA|NUMERO|        BAIRRO|      CEP| PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|GASOLINA| 03/03/2025|       5.65|    R$ / litro|  BRANCA|
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|  ETANOL| 03/03/2025|       3.78|    R$ / litro|  BRANCA|
|    NE|    AL|ARAPIRACA|AUTO POSTO M M GA...| 08.738.994/0001-50|      RODOVIA AL-220|  5848|   BOM SUCESSO|57309-

None

In [105]:
len(dados.columns), dados.count(), len(dados_2.columns), dados_2.count(), len(dados_3.columns), dados_3.count()

(14, 46509, 14, 44290, 14, 51685)

In [108]:
display(dados.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados.columns]).show())

+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|RUA|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|  0|     0|     0|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+



None

In [107]:
dados = dados.na.drop()

In [110]:
display(dados_2.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_2.columns]).show())

+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|RUA|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|  0|     3|    91|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+



None

In [111]:
display(dados_3.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_3.columns]).show())

+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|RUA|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|  0|     9|   114|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+



None

In [112]:
dados_2 = dados_2.na.drop()
dados_3 = dados_3.na.drop()

In [113]:
dados_2.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_2.columns]).show()

+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|RUA|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|  0|     0|     0|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+



In [114]:
dados_3.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_3.columns]).show()

+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|RUA|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|  0|     0|     0|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+---+------+------+---+-------+-----------+-----------+--------------+--------+



In [115]:
display(dados_2.show(5))

+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|      MUNICIPIO|             REVENDA|               CNPJ|                 RUA|NUMERO|    BAIRRO|      CEP|           PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|          GASOLINA| 03/02/2025|       7.27|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|GASOLINA ADITIVADA| 03/02/2025|       7.73|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...| 01

None

In [116]:
display(dados_3.show(5))

+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                 RUA|NUMERO|        BAIRRO|      CEP| PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|GASOLINA| 03/03/2025|       5.65|    R$ / litro|  BRANCA|
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|  ETANOL| 03/03/2025|       3.78|    R$ / litro|  BRANCA|
|    NE|    AL|ARAPIRACA|AUTO POSTO M M GA...| 08.738.994/0001-50|      RODOVIA AL-220|  5848|   BOM SUCESSO|57309-

None

## **Salvando dados na Camada Silver na Amazon S3**

In [121]:
dados.write.mode('overwrite').format('parquet').save('s3a://datalake-combustiveis-br/Silver/preco-gasolina-etanol-01')

In [122]:

dados_2.write.mode("overwrite").format('parquet').save("s3a://datalake-combustiveis-br/Silver/preco-gasolina-etanol-02")
dados_3.write.mode("overwrite").format('parquet').save("s3a://datalake-combustiveis-br/Silver/preco-gasolina-etanol-03")